I ran this query via the Datathon intro colab page then have used a CSV to load into a Pandas DataFrame

In [ ]:
ventAnalysis = client.query('''
    SELECT
      person_id,
      visit_occurrence_id,
      measurement_concept_id,
      measurement_datetime,
      measurement_type_concept_id,
      value_as_number,
      unit_concept_id,
      provider_id
    FROM measurement
    WHERE
      NOT provider_id IS NULL -- ignore unvalidated device data
      AND measurement_concept_id IN (
        3020716, --fio2
        42527120, --eMV
        42527121, --iMV
        21490855, --PEEP
        21490582, --Cdyn
        21490583, --Cstat
        42527086, --mean airway pressure
        44782825, --Pplat
        3035357, --ETCO2
        3013290, --pCO2
        3027315 --pO2

    )
    ''',
                            job_config=job_config).to_dataframe()

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/Users/matthewwillis/Local Dev/ESICM_datathon/out.csv')

In [7]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [15]:
# bin results into 20 bins
pd.cut(df['value_as_number'], 20).value_counts().sort_index()

value_as_number
(-1.005, 50.25]     5107128
(50.25, 100.5]         5904
(100.5, 150.75]        1994
(150.75, 201.0]         490
(201.0, 251.25]          12
(251.25, 301.5]           4
(301.5, 351.75]          12
(351.75, 402.0]          26
(402.0, 452.25]          38
(452.25, 502.5]          70
(502.5, 552.75]          52
(552.75, 603.0]          18
(603.0, 653.25]          14
(653.25, 703.5]           4
(703.5, 753.75]           6
(753.75, 804.0]           0
(804.0, 854.25]           0
(854.25, 904.5]           0
(904.5, 954.75]           0
(954.75, 1005.0]          2
Name: count, dtype: int64

In [31]:
# remove outliers (MV < 100 and MV > 0)

df = df[df['value_as_number'] < 100]
df = df[df['value_as_number'] > 0]

df.reset_index(drop=True, inplace=True)
df

,person_id,visit_occurrence_id,measurement_concept_id,measurement_datetime,measurement_type_concept_id,value_as_number,unit_concept_id,provider_id
0,18392,21299,42527120,2006-01-01 03:45:00+00:00,32817,1.0,8698,40
1,18979,21980,42527120,2013-01-07 18:30:00+00:00,32817,1.0,8698,40
2,16354,18964,42527120,2006-01-01 00:11:00+00:00,32817,2.0,8698,40
3,19498,22568,42527120,2006-01-24 15:39:00+00:00,32817,2.0,8698,40
4,15547,18007,42527120,2006-01-01 04:29:00+00:00,32817,2.0,8698,40
...,...,...,...,...,...,...,...,...
5106687,6229,7215,42527120,2013-01-10 21:39:00+00:00,32817,0.9,8698,40
5106688,5803,6719,42527120,2015-08-08 01:42:00+00:00,32817,0.9,8698,40
5106689,6096,7060,42527120,2006-01-09 22:22:00+00:00,32817,0.9,8698,40
5106690,3030,3488,42527120,2006-01-19 19:17:00+00:00,32817,0.9,8698,40


In [66]:
# group by person and occurrence
df_grouped = df.groupby(['person_id', 'visit_occurrence_id']).agg({'measurement_datetime': ['max', 'min'], 'value_as_number': ['count', 'mean', 'std']})

In [67]:
df_grouped.columns

MultiIndex([('measurement_datetime',   'max'),
            ('measurement_datetime',   'min'),
            (     'value_as_number', 'count'),
            (     'value_as_number',  'mean'),
            (     'value_as_number',   'std')],
           )

In [68]:
# calculate the difference between the max and min dates (for Minute Ventilation) - rough idea of duration of mechanical ventilation

df_grouped[('measurement_datetime','max')] = pd.to_datetime(df_grouped[('measurement_datetime','max')])
df_grouped[('measurement_datetime','min')] = pd.to_datetime(df_grouped[('measurement_datetime','min')])

df_grouped['diff'] = (df_grouped[('measurement_datetime','max')] - df_grouped[('measurement_datetime','min')]).dt.days
df_grouped[df_grouped['diff'] >= 4]

measurement_datetime  \
                                                    max   
person_id visit_occurrence_id                             
14        15                  2013-01-10 18:38:00+00:00   
20        21                  2006-01-06 20:49:00+00:00   
31        32                  2013-01-08 17:51:00+00:00   
33        35                  2013-01-06 09:00:00+00:00   
36        38                  2013-01-30 23:25:00+00:00   
...                                                 ...   
20271     23488               2013-01-06 08:31:00+00:00   
20284     23503               2013-01-06 06:43:00+00:00   
20289     23509               2013-07-26 17:58:00+00:00   
20290     23510               2013-01-05 04:40:00+00:00   
20293     23516               2018-01-05 18:29:00+00:00   

                                                        value_as_number  \
                                                    min           count   
person_id visit_occurrence_id                                             
14        15                  2013-01-01 01:38:00+00:00             840   
20        21                  2006-01-01 03:07:00+00:00             564   
31        32                  2013-01-01 00:06:00+00:00             728   
33        35                  2013-01-01 00:29:00+00:00             452   
36        38                  2013-01-01 00:03:00+00:00            2540   
...                                                 ...             ...   
20271     23488               2013-01-01 00:31:00+00:00             484   
20284     23503               2013-01-01 00:43:00+00:00             492   
20289     23509               2013-07-16 12:43:00+00:00             960   
20290     23510               2013-01-01 04:15:00+00:00             350   
20293     23516               2017-12-19 17:55:00+00:00             828   

                                                   diff  
                                    mean       std       
person_id visit_occurrence_id                            
14        15                    9.684524  2.237618    9  
20        21                    6.749645  2.128004    5  
31        32                    7.312088  4.281937    7  
33        35                    7.258850  1.237205    5  
36        38                    8.097795  3.681198   29  
...                                  ...       ...  ...  
20271     23488                10.357438  4.802704    5  
20284     23503                 9.873577  2.445161    5  
20289     23509                11.253333  3.570226   10  
20290     23510                11.621714  7.359612    4  
20293     23516                17.751208  4.137122   17  

[3488 rows x 6 columns]

In [69]:
# as expected data skewed toward shorter stays
pd.cut(df_grouped['diff'], 20).value_counts().sort_index()

diff
(-0.237, 11.85]    13973
(11.85, 23.7]        971
(23.7, 35.55]        334
(35.55, 47.4]        139
(47.4, 59.25]         75
(59.25, 71.1]         30
(71.1, 82.95]         11
(82.95, 94.8]          4
(94.8, 106.65]         3
(106.65, 118.5]        0
(118.5, 130.35]        1
(130.35, 142.2]        0
(142.2, 154.05]        0
(154.05, 165.9]        0
(165.9, 177.75]        0
(177.75, 189.6]        2
(189.6, 201.45]        0
(201.45, 213.3]        1
(213.3, 225.15]        0
(225.15, 237.0]        1
Name: count, dtype: int64

In [72]:
df_grouped[df_grouped['diff'] >= 4]['diff'].describe()

count    3488.000000
mean       15.034690
std        14.587049
min         4.000000
25%         6.000000
50%        10.000000
75%        19.000000
max       237.000000
Name: diff, dtype: float64

In [73]:
df_vac = pd.read_csv('/Users/matthewwillis/Local Dev/ESICM_datathon/out_vent.csv')

df_vac.columns

Index(['Unnamed: 0', 'person_id', 'visit_occurrence_id',
       'measurement_concept_id', 'measurement_datetime',
       'measurement_type_concept_id', 'value_as_number', 'unit_concept_id',
       'provider_id'],
      dtype='object')

In [92]:
concept_ids = {
         "fio2":3020716,
        "eMV":42527120,
        "iMV":42527121,
        "PEEP":21490855,
        "Cdyn":21490582,
        "Cstat":21490583,
        "mean airway pressure":42527086,
        "Pplat":44782825,
        "ETCO2":3035357,
        "pCO2":3013290,
        "pO2":3027315
}

concept_ids['fio2']

3020716

In [83]:
for_series = []

for i in range(0, len(df_grouped[df_grouped['diff'] >= 4])):
    for_series.append(df_grouped[df_grouped['diff'] >= 4].index[i][1])


df_vented_pts = pd.Series(for_series, index=for_series)

df_vented_pts

15          15
21          21
32          32
35          35
38          38
         ...  
23488    23488
23503    23503
23509    23509
23510    23510
23516    23516
Length: 3488, dtype: int64

In [100]:
df_fio2 = df_vac[df_vac['measurement_concept_id'] == concept_ids['fio2'] & ].drop(['Unnamed: 0', 'person_id', 'unit_concept_id', 'provider_id','measurement_concept_id', 'measurement_type_concept_id'], axis=1).sort_values(['visit_occurrence_id', 'measurement_datetime'])

In [111]:
df_fio2['measurement_datetime'] = pd.to_datetime(df_fio2['measurement_datetime'])

df_fio2_grouped = df_fio2.groupby(['visit_occurrence_id', pd.Grouper(key='measurement_datetime', freq='D')]).agg({'value_as_number': ['max', 'min']})

In [116]:
list_uniques = []

for i in range(0, len(df_fio2_grouped)):
    list_uniques.append(df_fio2_grouped.index[i][0])

new_series = pd.Series(list_uniques)

len(new_series.unique())

914

In [117]:
df_fio2_grouped

value_as_number      
                                                          max   min
visit_occurrence_id measurement_datetime                           
12                  2006-01-01 00:00:00+00:00            98.0  22.0
                    2006-01-02 00:00:00+00:00            51.0  40.0
                    2006-01-03 00:00:00+00:00            41.0  40.0
                    2006-01-04 00:00:00+00:00            73.0  40.0
                    2006-01-05 00:00:00+00:00            40.0  40.0
...                                                       ...   ...
23474               2006-01-04 00:00:00+00:00            36.0  31.0
                    2006-01-05 00:00:00+00:00            99.0  31.0
                    2006-01-06 00:00:00+00:00            32.0  31.0
23475               2006-01-01 00:00:00+00:00            39.0  38.0
23506               2006-01-01 00:00:00+00:00            57.0  40.0

[4125 rows x 2 columns]